<a href="https://colab.research.google.com/github/HanJunSeoJB/security/blob/master/LSTM_Model_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import sys
from pathlib import Path
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from itertools import chain
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed, Dense


def dataframe_from_csv(target):
    return pd.read_csv(target).rename(columns=lambda x: x.strip())

def dataframe_from_csvs(targets):
    return pd.concat([dataframe_from_csv(x) for x in targets])

TEST_DATASET = sorted([x for x in Path("/content/drive/MyDrive/test").glob("*.csv")])
#TRAIN_DATASET = sorted([x for x in Path("./test/").glob("*.csv")])
TEST_DF_RAW = dataframe_from_csvs(TEST_DATASET)
ATTACK_DF = TEST_DF_RAW['attack']


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def extract_sequences_with_context_and_return_list(df):
    data = df['attack'].to_numpy().flatten()

    flag = 0
    start_index = None
    start_index_list = []
    end_index_list = []
    list_size = []

    for index, value in enumerate(data):
        if value == 1 and flag == 0:
            start_index = index
            start_index_list.append(start_index)
            flag = 1
        elif value == 0 and flag == 1:
            end_index = index
            end_index_list.append(end_index)
            sequence_length = end_index - start_index
            list_size.append(sequence_length)
            print(f"Sequence length: {sequence_length}")
            flag = 0


    if flag == 1:
        end_index = len(data)
        end_index_list.append(end_index)
        sequence_length = end_index - start_index
        list_size.append(sequence_length)
        print(f"Sequence length: {sequence_length}")

    return start_index_list, end_index_list, list_size



In [ ]:
start_list = []
end_list = []
list_size = []

start_list, end_list, list_size = extract_sequences_with_context_and_return_list(TEST_DF_RAW)
print(len(start_list), len(end_list))

Sequence length: 559
Sequence length: 309
Sequence length: 561
Sequence length: 260
Sequence length: 358
Sequence length: 331
Sequence length: 511
Sequence length: 255
Sequence length: 513
Sequence length: 365
Sequence length: 179
Sequence length: 151
Sequence length: 346
Sequence length: 513
Sequence length: 344
Sequence length: 365
Sequence length: 630
Sequence length: 346
Sequence length: 508
Sequence length: 567
Sequence length: 377
Sequence length: 179
Sequence length: 564
Sequence length: 564
Sequence length: 305
Sequence length: 513
Sequence length: 552
Sequence length: 513
Sequence length: 386
Sequence length: 397
Sequence length: 378
Sequence length: 290
Sequence length: 345
Sequence length: 346
Sequence length: 2888
Sequence length: 335
Sequence length: 312
Sequence length: 312
38 38


In [ ]:
def concateList(df, start_list, end_list, list_size):
  segments = []
  for start, end, size in zip(start_list, end_list, list_size):
        segment = df[start-size:end + 1]
        segments.append(segment)

  concatenated_df = pd.concat(segments, axis=0, ignore_index=True)
  return concatenated_df

In [8]:
DROP_FIELD = [",time", "attack_P1", "attack_P2", "attack_P3","attack"]
VALID_COLUMNS_IN_TRAIN_DATASET = TEST_DF_RAW.columns.drop(DROP_FIELD) # DROP_FIELD를 통해 normalization에 사용하지 않을 변수를 제거함.
TAG_MIN = TEST_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].min()
TAG_MAX = TEST_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].max()

In [9]:
def normalize(df, TAG_MIN, TAG_MAX):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]:
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c])
    return ndf

# Min-Max Normalize
TEST_DF = normalize(TEST_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET], TAG_MIN, TAG_MAX).ewm(alpha=0.9).mean()


In [ ]:
def boundary_check(df):
    x = np.array(df, dtype=np.float32)
    is_string = df.applymap(lambda x: isinstance(x, str))
    return np.any(x > 1.0), np.any(x < 0), np.any(np.isnan(x)), np.any(is_string, axis=1)

# Boundary Check
print(boundary_check(TEST_DF))

(False, False, False, 0         False
1         False
2         False
3         False
4         False
          ...  
152995    False
152996    False
152997    False
152998    False
152999    False
Length: 444600, dtype: bool)


In [ ]:
#준지도학습

window_size = 60
label_size = 30000
def sliding_window2(df):
    features = []
    targets = []
    for i in range(0, len(df) - window_size + 1):
        # 현재 슬라이스를 직접 리스트로 변환합니다.
        window = df[i:i+window_size]
        target = 1 if np.any(window['attack'] == 1) else 0
        feature = window[['P1_B2004', 'P1_B2016', 'P1_B3004', 'P1_B3005', 'P1_B4002', 'P1_B4005', 'P1_B400B',
        'P1_B4022', 'P1_FCV01D', 'P1_FCV01Z', 'P1_FCV02D', 'P1_FCV02Z', 'P1_FCV03D',
        'P1_FCV03Z', 'P1_FT01', 'P1_FT01Z', 'P1_FT02', 'P1_FT02Z', 'P1_FT03', 'P1_FT03Z',
        'P1_LCV01D', 'P1_LIT01', 'P1_PCV01D', 'P1_PCV01Z', 'P1_PCV02D', 'P1_PCV02Z',
        'P1_PIT01', 'P1_PIT02', 'P1_TIT01', 'P1_TIT02']].values
        features.append(feature)
        targets.append(target)
    features = np.array(features, dtype=np.float32)
    targets = np.array(targets)
    return features, targets

features = []
targets = []


In [ ]:
df = concateList(TEST_DF_RAW, start_list, end_list, list_size)
features, targets = sliding_window2(df)
print(features.shape)
print(targets.shape)

(35033, 60, 30)
(35033,)


In [26]:
#비지도 학습

window_size = 60
label_size = 100000
def sliding_window_unsupervised(df, window_size, feature_columns, answer_column):
    # 데이터 프레임에서 필요한 컬럼만 먼저 추출하여 NumPy 배열로 변환
    data = df[feature_columns].values
    answers = df.values

    num_samples = len(df) - window_size
    features = np.empty((num_samples, window_size, len(feature_columns)), dtype=np.float32)
    targets = np.empty((num_samples, len(feature_columns)), dtype=np.float32)
    answer_targets = np.empty(num_samples, dtype=int)

    for i in range(num_samples):
        features[i] = data[i:i+window_size]
        targets[i] = data[i+window_size]
        answer_targets[i] = 1 if np.any(answers[i:i+window_size] == 1) else 0

    return features, targets, answer_targets

features = []
targets = []
answers = []

In [27]:
feature_columns = ['P1_B2004', 'P1_B2016', 'P1_B3004', 'P1_B3005', 'P1_B4002', 'P1_B4005', 'P1_B400B',
                   'P1_B4022', 'P1_FCV01D', 'P1_FCV01Z', 'P1_FCV02D', 'P1_FCV02Z', 'P1_FCV03D',
                   'P1_FCV03Z', 'P1_FT01', 'P1_FT01Z', 'P1_FT02', 'P1_FT02Z', 'P1_FT03', 'P1_FT03Z',
                   'P1_LCV01D', 'P1_LIT01', 'P1_PCV01D', 'P1_PCV01Z', 'P1_PCV02D', 'P1_PCV02Z',
                   'P1_PIT01', 'P1_PIT02', 'P1_TIT01', 'P1_TIT02']

features, targets, answers = sliding_window_unsupervised(TEST_DF_RAW[:label_size], 60, feature_columns, ATTACK_DF[:label_size])
print(features.shape)
print(targets.shape)
print(answers.shape)

(99940, 60, 30)
(99940, 30)
(99940,)


In [ ]:
features, targets = sliding_window2(TEST_DF_RAW[:label_size])
print(features.shape)
print(targets.shape)

In [17]:
# 데이터 분할
features_train, features_test, _, labels_test = train_test_split(features, answers, test_size=0.2,random_state=42)

In [28]:
def build_autoencoder(window_size, num_features):
    model = Sequential()
    # 인코더
    model.add(LSTM(100, activation='relu', input_shape=(window_size, num_features), return_sequences=False))
    model.add(Dense(num_features))  # 잠재 공간 표현
    model.add(RepeatVector(window_size))  # 디코더로 전달할 시퀀스 재생성

    # 디코더
    model.add(LSTM(100, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(num_features)))

    # 옵티마이저에 학습률 설정
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mae')
    return model

In [30]:
# 모델 생성
autoencoder = build_autoencoder(window_size, len(feature_columns))

In [31]:
# 모델 훈련
autoencoder.fit(features_train, features_train, epochs=50, batch_size=64, verbose=1)

Epoch 1/50
1250/1250 [==============================] - 247s 179ms/step - loss: nan
Epoch 2/50
1002/1250 [=======================>......] - ETA: 43s - loss: nan

KeyboardInterrupt: 

In [ ]:
# 모델 정의
model = Sequential([
    LSTM(50, input_shape=(60, 30), return_sequences=True),  # 첫 번째 LSTM 층
    Dropout(0.2),  # 과적합 방지를 위한 드롭아웃
    LSTM(50),  # 두 번째 LSTM 층
    Dropout(0.2),  # 과적합 방지를 위한 드롭아웃
    Dense(1, activation='sigmoid')  # 출력 층, 이진 분류를 위한 시그모이드 활성화 함수
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# EarlyStopping 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
# 모델 학습
history = model.fit(features_train, labels_train, epochs=10, batch_size=64, validation_data=(features_test, labels_test), verbose=1)

Train on 28026 samples, validate on 7007 samples
Epoch 1/10
28026/28026 [==============================] - 47s 2ms/sample - loss: 0.5332 - acc: 0.7030 - val_loss: 0.4949 - val_acc: 0.7237
Epoch 2/10
28026/28026 [==============================] - 44s 2ms/sample - loss: 0.4945 - acc: 0.7299 - val_loss: 0.5028 - val_acc: 0.7159
Epoch 3/10
28026/28026 [==============================] - 43s 2ms/sample - loss: 0.5523 - acc: 0.6850 - val_loss: 0.5506 - val_acc: 0.6897
Epoch 4/10
28026/28026 [==============================] - 43s 2ms/sample - loss: 0.5319 - acc: 0.6913 - val_loss: 0.5492 - val_acc: 0.6925
Epoch 5/10
28026/28026 [==============================] - 43s 2ms/sample - loss: 0.5414 - acc: 0.6825 - val_loss: 0.5209 - val_acc: 0.6973
Epoch 6/10
28026/28026 [==============================] - 44s 2ms/sample - loss: 0.5189 - acc: 0.7037 - val_loss: 0.5151 - val_acc: 0.7067
Epoch 7/10
28026/28026 [==============================] - 44s 2ms/sample - loss: 0.5149 - acc: 0.7085 - val_loss: 0.5

In [ ]:
# 예측 수행
predictions = model.predict(features_test)
predictions = (predictions > 0.5).astype(int)

# 정확도 계산
acc = accuracy_score(labels_test, predictions)
print("Accuracy:", acc)

# F1 스코어 계산
f1 = f1_score(labels_test, predictions)
print("F1 Score:", f1)

# 혼동 행렬 계산
cm = confusion_matrix(labels_test, predictions)
print("Confusion Matrix:\n", cm)

In [ ]:
# 모델로부터 예측값을 얻습니다.
predicted = autoencoder.predict(X_test)

# 각 피쳐별 이상 여부 판단 (임계값은 예를 들어 0.01로 가정)
threshold = 0.01
anomalies = np.abs(predicted - X_test) > threshold

# 이상 여부 판단 결과를 이진 플래그로 변환 (피처 중 하나라도 이상이면 이상으로 판단)
anomaly_flags = np.any(anomalies, axis=1).astype(int)

# 실제 공격 레이블과의 비교
print("Classification Report:")
print(classification_report(labels_test, anomaly_flags))

print("Accuracy Score:")
print(accuracy_score(labels_test, anomaly_flags))

print("F1 Score:")
print(f1_score(labels_test, anomaly_flags, average='macro'))  # 'macro', 'micro', 'weighted' 등 필요에 따라 조정

# 컨퓨전 매트릭스 생성 및 시각화
cm = confusion_matrix(labels_test, anomaly_flags)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Anomaly'], yticklabels=['Normal', 'Anomaly'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()